[Reference](https://medium.com/@thoren.lederer/use-pandas-dataframe-with-sql-support-as-a-network-database-thanks-to-duckdb-2a46e1f88b66)

In [1]:
pip install yfinance

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

In [2]:
import yfinance as yf

# download a small dataset
prices_df = yf.download(tickers=["QQQ", "NVDA", "AMD", "AAPL", "TSLA", "MSFT"], period='60d', interval='5m')
prices_df = prices_df.stack()
prices_df.index.names = ['date', 'symbol']

[*********************100%%**********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['AAPL', 'TSLA', 'MSFT', 'AMD', 'QQQ', 'NVDA']: YFInvalidPeriodError("%ticker%: Period '60d' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']")


In [10]:
!pip install chipmunkdb-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26306 sha256=f775dc50c7fcf27370efdd318ac0e1d804689d20340d0511d651f4ba8e2d75c9
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing


In [3]:
import time
from chipmunkdb.ChipmunkDb import ChipmunkDb
chipmunkDb = ChipmunkDb("localhost")

# lets calculate the time to save the dataframe to chipmunkdb
start = time.time()
chipmunkDb.save_as_pandas(prices_df, "stock_prices")
print("Time to save: ", time.time()-start, " seconds")

Time to save:  0.0005600452423095703  seconds


In [4]:
# lets than read the dataframe from chipmunkdb
import time
from chipmunkdb.ChipmunkDb import ChipmunkDb
chipmunkDb = ChipmunkDb("localhost")

# lets read it back and calulate the time
start = time.time()
df = chipmunkDb.collection_as_pandas("stock_prices")
print("Time to load: ", time.time()-start, " seconds")

Error:  HTTPConnectionPool(host='localhost', port=8091): Max retries exceeded with url: /collection/stock_prices/rawStream (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7843097b5000>: Failed to establish a new connection: [Errno 111] Connection refused'))
Time to load:  0.012173175811767578  seconds


In [6]:
from chipmunkdb.ChipmunkDb import ChipmunkDb
# lets query only the symbols "NVDA" and "AAPL" for the last 10 rows
start = time.time()
results = chipmunkDb.query("select index_symbol, LAST(Close) from stock_prices WHERE index_symbol IN ('AAPL', 'NVDA') group by index_symbol ")
print("Time to query: ", time.time()-start, " seconds")

Error:  HTTPConnectionPool(host='localhost', port=8091): Max retries exceeded with url: /query?q=select%20index_symbol,%20LAST(Close)%20from%20stock_prices%20WHERE%20index_symbol%20IN%20('AAPL',%20'NVDA')%20group%20by%20index_symbol%20 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7843095800a0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Time to query:  0.003989696502685547  seconds
